In [15]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn import model_selection
import warnings

In [2]:
class logistic_regression:
    def __init__(self, n_iterations=1000, eta=0.05):
        self.n_iterations = 1000
        self.eta = eta

    @staticmethod
    def log_grad(w, c, X, target):
        m = X.shape[0]
        y = (2 * target - 1)
        score = np.dot(X, w.T).flatten()
        Z = -y / (m * (1 + np.exp(y * score)))
        grad = Z[np.newaxis, :].dot(X)
        return grad / m, np.sum(Z) / m

    @classmethod
    def optimize(cls, w, c, X, y, n_iterations, eta):
        for i in range(n_iterations):
            grad_w, grad_c = cls.log_grad(w, c, X, y)
            w = w - eta * grad_w
            c = c - eta * grad_c
        return w, c

    def fit(self, X, y):
        self.X = X
        self.y_true = y
        w0 = np.zeros((1, X.shape[1]))
        c0 = 0
        self.w, self.c = self.optimize(w0, c0, X, y, self.n_iterations, self.eta)

    def predict_proba(self, X):
        score = X.dot(self.w.T).flatten() + self.c
        self.score = score
        return 1 / (1 + np.exp(-score))

    def predict(self, X, thr=0.5):
        proba = self.predict_proba(X)
        y_predicted = np.zeros(proba.shape, dtype=bool)
        y_predicted[proba > thr] = 1
        y_predicted[proba <= thr] = 0
        self.y_predicted = y_predicted
        return y_predicted

In [3]:
def get_error(pred, y):
    return np.sum((pred != y).astype(int)) / len(y)

In [4]:
def adaboost(X, y, N):

    # Размер выборки
    n_objects = len(X)

    # Запишем количество классов в переменную
    n_classes = len(np.unique((y)))

    # Начальные веса деревьев
    w = np.ones(n_objects) / n_objects

    # Деревья с весами будем записывать в список
    models = []

    for n in range(N):
        # Зададим дерево и обучим его
        clf = DecisionTreeClassifier(max_depth=1)
        clf.fit(X, y, w)

        predictions = clf.predict(X)
        e = get_error(predictions, y)
        # отбросим дерево, если его ошибка больше 0.5
        # Запишем условие в общем виде (применимо к небинарным классификаторам)
        if e >= 1 - 1/n_classes: 
            break

        # Вычислим вес для дерева
        alpha = 0.5 * np.log((1 - e) / e)

        # Найдем индексы правильно классифицированных элементов
        match = predictions == y

        # Увеличим веса для неправильно классифицированных элементов
        w[~match] *= np.exp(alpha)

        # Нормализуем веса
        w /= w.sum()

        # Добавим дерево с весом в список
        models.append((alpha, clf))
    
    return models

In [5]:
def log_loss(pred, y):
    return -np.sum(y*np.log(pred)+(1-y)*np.log(1-pred))/len(y)

In [22]:
def adaboost_2(X, y, N):
    n_objects = len(X)
    n_classes = len(np.unique((y)))
    w = np.ones(n_objects) / n_objects
    c0 = 0
    models = []
    for n in range(N):
        llr = logistic_regression()
        llr.fit(X,y)
        
        predictions = llr.predict(X)
        error = log_loss(llr.predict_proba(X), y)
        #if error >= 1 - 1/n_classes:
            #break
            
        alpha = 0.5 * np.log((1 - error) / error)
        match = predictions == y
        w[~match] *= np.exp(alpha)
        w /= w.sum()
        models.append((alpha, llr))
    return models

In [7]:
n_objects = len(X_train)
n_classes = len(np.unique((y_train)))
w = np.ones(n_objects) / n_objects
c0 = 0

NameError: name 'X_train' is not defined

In [44]:
n_objects

426

In [45]:
n_classes

2

In [46]:
llr = logistic_regression()

In [47]:
llr.fit(X_train,y_train)

In [48]:
predictions = llr.predict(X_train)
predictions

array([False,  True,  True,  True, False,  True,  True,  True,  True,
        True, False,  True,  True,  True, False,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True, False,  True,
        True,  True, False, False,  True,  True,  True,  True, False,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True, False,  True,
        True, False,  True,  True, False,  True,  True, False, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True, False,  True,  True, False,  True, False,  True,  True,
        True,  True,  True, False, False,  True,  True, False, False,
        True,  True, False,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True, False,

In [49]:
error = log_loss(llr.predict_proba(X_train), y_train)
error

2.5701152734435806

In [17]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)

In [18]:
N = 50

models = adaboost(X_train, y_train, N)

In [19]:
def predict(X, models):
    
    n_classes = 2
    n_objects = len(X)
    
    # вначале обозначим предсказание нулевым массивом
    y_pred = np.zeros((n_objects, n_classes))
    
    for alpha, clf in models:
        prediction = clf.predict(X)
        # Для каждого предсказания будем прибавлять alpha к
        # элементу с индексом предсказанного класса
        y_pred[range(n_objects), prediction] += alpha
    
    # выберем индексы с максимальными суммарными весами -
    # получим предсказанные алгоритмом классы
    y_pred = np.argmax(y_pred, axis=1)
    
    return y_pred

print(f'Точность алгоритма на обучающей выборке: {(1 - get_error(predict(X_train, models), y_train)) * 100:.3f}')

Точность алгоритма на обучающей выборке: 96.714


In [20]:
print(f'Точность алгоритма на тестовой выборке: {(1 - get_error(predict(X_test, models), y_test)) * 100:.3f}')

Точность алгоритма на тестовой выборке: 94.406


In [21]:
models_2 = adaboost_2(X_train, y_train, N)

AttributeError: 'str' object has no attribute 'decode'

In [13]:
models_2

[(0.12536786463224345, <__main__.logistic_regression at 0x1b2254c8ca0>),
 (0.12536786463224345, <__main__.logistic_regression at 0x1b2254c8430>),
 (0.12536786463224345, <__main__.logistic_regression at 0x1b2254c8970>),
 (0.12536786463224345, <__main__.logistic_regression at 0x1b2254c8280>),
 (0.12536786463224345, <__main__.logistic_regression at 0x1b2254c8190>),
 (0.12536786463224345, <__main__.logistic_regression at 0x1b2254c8460>),
 (0.12536786463224345, <__main__.logistic_regression at 0x1b2254c8c70>),
 (0.12536786463224345, <__main__.logistic_regression at 0x1b205cea6d0>),
 (0.12536786463224345, <__main__.logistic_regression at 0x1b205cea790>),
 (0.12536786463224345, <__main__.logistic_regression at 0x1b205ceafa0>),
 (0.12536786463224345, <__main__.logistic_regression at 0x1b205cead30>),
 (0.12536786463224345, <__main__.logistic_regression at 0x1b205cea550>),
 (0.12536786463224345, <__main__.logistic_regression at 0x1b2264e6100>),
 (0.12536786463224345, <__main__.logistic_regressio

ВОТ ТУТ ниже ошибка 

boolean index did not match indexed array along dimension 1; dimension is 2 but corresponding boolean dimension is 426

In [16]:
def predict_2(X, models):
    
    n_classes = 2
    n_objects = len(X)
    
    # вначале обозначим предсказание нулевым массивом
    y_pred = np.zeros((n_objects, n_classes))
    
    for alpha, llr in models:
        prediction = llr.predict(X)
        # Для каждого предсказания будем прибавлять alpha к
        # элементу с индексом предсказанного класса
        y_pred[range(n_objects), prediction] += alpha
    
    # выберем индексы с максимальными суммарными весами -
    # получим предсказанные алгоритмом классы
    y_pred = np.argmax(y_pred, axis=1)
    
    return y_pred

print(f'Точность алгоритма на обучающей выборке: {(1 - log_loss(predict_2(X_train,models_2), y_train)) * 100:.3f}')

IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
print(f'Точность алгоритма на тестовой выборке: {(1 - log_loss(predict(X_test, models_2), y_test)) * 100:.3f}')

In [13]:
n_classes = 2
n_objects = len(X_train)
y_pred = np.zeros((n_objects, n_classes))
y_pred.shape

(426, 2)

In [14]:
llr = logistic_regression()
llr.fit(X,y)
prediction = llr.predict(X_train)
prediction.shape

(426,)

array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True, False, False,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True, False,  True, False,  True,  True, False, False, False,
        True,  True,  True,  True,  True,  True, False, False,  True,
        True, False, False,  True, False,  True,  True,  True,  True,
        True,  True, False,  True, False,  True, False,  True,  True,
       False,  True, False, False, False,  True, False,  True,  True,
        True,  True,  True, False, False, False, False,  True,  True,
        True, False,  True,  True,  True, False, False, False,  True,
        True, False,  True, False,  True,  True,  True,  True,  True,
       False,  True, False,  True,  True,  True,  True, False,  True,
        True,  True, False,  True,  True, False, False,  True, False,
        True,  True,  True,  True, False,  True,  True, False,  True,
        True,  True,

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

In [ ]:
X_train.shape

In [23]:
llr = logistic_regression()
llr.fit(X_train,y_train)

In [24]:

predictions = llr.predict(X_train)
predictions.shape

(426,)

In [25]:
error = log_loss(llr.predict_proba(X_train), y_train)
error

0.7495366726443706

In [26]:
match = predictions == y_train
match

array([ True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False,  True,  True,  True,  True,  True,
        True, False,

In [27]:
match.shape

(426,)

In [28]:
alpha = 0.5 * np.log((1 - error) / error)

In [29]:
alpha

-0.5480713668993292

In [30]:
n_objects = len(X_train)
n_objects

426

In [31]:
w = np.ones(n_objects) / n_objects
w.shape

(426,)

In [32]:
w[~match] *= np.exp(alpha)

In [33]:
w

array([0.00234742, 0.00234742, 0.00234742, 0.00135696, 0.00234742,
       0.00234742, 0.00234742, 0.00234742, 0.00234742, 0.00234742,
       0.00234742, 0.00234742, 0.00234742, 0.00234742, 0.00234742,
       0.00234742, 0.00234742, 0.00234742, 0.00234742, 0.00234742,
       0.00234742, 0.00234742, 0.00234742, 0.00234742, 0.00234742,
       0.00234742, 0.00234742, 0.00234742, 0.00234742, 0.00234742,
       0.00234742, 0.00135696, 0.00234742, 0.00234742, 0.00234742,
       0.00234742, 0.00234742, 0.00234742, 0.00234742, 0.00234742,
       0.00135696, 0.00234742, 0.00234742, 0.00234742, 0.00234742,
       0.00234742, 0.00234742, 0.00234742, 0.00234742, 0.00234742,
       0.00234742, 0.00135696, 0.00234742, 0.00234742, 0.00234742,
       0.00234742, 0.00234742, 0.00234742, 0.00234742, 0.00234742,
       0.00234742, 0.00234742, 0.00234742, 0.00135696, 0.00234742,
       0.00234742, 0.00234742, 0.00234742, 0.00234742, 0.00234742,
       0.00234742, 0.00234742, 0.00234742, 0.00234742, 0.00234